# 1. Initialisation

Importations

In [965]:
import numpy as np
import matplotlib
matplotlib.use("TkAgg")  # Ou "Qt5Agg"
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
from scipy.interpolate import lagrange
from scipy.ndimage import gaussian_filter1d

Paramètres du modèle

In [966]:
SIM_DURATION = 1000

WINDOW_SIZE = 20
DISTANCE = 5
BEAM_ANGLE = 0.5

DT = 1
NOISE = 0.1

RESOLUTION = 0.05
RADIUS_DECREASE_COEF = 0.5
RADIUS_INCREASE_COEF = 2

# STEP_COEF <= (1 + RADIUS_INCREASE_COEF)/np.sqrt(5)
STEP_COEF = 1.25

beam_radius = 0
power = 0

count_frame_found = 0
count_frame_tracking = 0

In [967]:
# target_position = np.array([WINDOW_SIZE/2, WINDOW_SIZE/2])
target_speed = np.array([0, 0])
# target_position = np.array([np.random.uniform(WINDOW_SIZE/4, 3*WINDOW_SIZE/4), np.random.uniform(WINDOW_SIZE/4, 3*WINDOW_SIZE/4)])
# target_speed = np.array([np.random.uniform(-0.1, 0.1), np.random.uniform(-0.1, 0.1)])
# target_position = np.array([WINDOW_SIZE, WINDOW_SIZE])    # Pour écarter l'AUV du centre
sensor_position = np.array([WINDOW_SIZE/2, WINDOW_SIZE/2])
tracking_position = np.copy(sensor_position)

is_tracking = False

# search_path = [(0, 0)] * SIM_DURATION

#Paramètres du déplacement du capteur
initial_position = tracking_position
initial_axis = 0


Initialisation du plot

In [968]:
fig, ax = plt.subplots()
ax.set_xlim(0, WINDOW_SIZE)
ax.set_ylim(0, WINDOW_SIZE)
target_line, = ax.plot([], [], 'r--', alpha=0.5, label="Trajectoire cible")
tracking_line, = ax.plot([], [], 'b--', alpha=0.5, label="Trajectoire suivi")
sensor_dot, = ax.plot([], [], 'go', markersize=8, label="Capteur")
target_dot, = ax.plot([], [], 'ro', markersize=8, label="Cible")
tracking_dot, = ax.plot([], [], 'bo', markersize=8, label="Tracking")
ax.legend()
ax.set_title("Simulation Pointing, Acquisition, and Tracking (PAT)")

Text(0.5, 1.0, 'Simulation Pointing, Acquisition, and Tracking (PAT)')

# 2. Génération de trajectoire

A. Paramètres

In [969]:
NUMBER_OF_TRAJ = 100       # Number of trajectories
MAX_DEG_X = 4  # Maximum degree for x(t)
MAX_DEG_Y = 3  # Maximum degree for y(t)

trajectory = []

B. Génération et affichage

In [970]:
def sample_z():
    """Renvoie une variable aléatoire de loi demi-cercle"""
    while True:
        z = np.random.uniform(-np.sqrt(2/np.pi), np.sqrt(2/np.pi))
        p_z = np.sqrt((2/np.pi) - z**2)
        if np.random.uniform(0, np.sqrt(2/np.pi)) <= p_z:
            return z


def generate_lagrange_polynomial(max_degree):
    """Renvoie un polynome interpolé de degré <=max_degree"""
    degree = np.random.choice(range(1, max_degree + 1))  # Randomly select the polynomial degree
    t_points = np.linspace(0, 500, degree + 1)
    values = WINDOW_SIZE * (np.pi / np.sqrt(2)) * np.array([sample_z() for _ in range(degree + 1)])
    values = (values + 2*WINDOW_SIZE) / 4
    return lagrange(t_points, values)


def add_noise_to_trajectories(trajectories, mu, sigma):
    """Ajoute un bruit gaussien à une trajectoire"""
    noisy_trajectories = []
    for trajectory in trajectories:
        t, x_t, y_t = trajectory
        noise_x = np.random.normal(mu, sigma, size=x_t.shape)
        noise_y = np.random.normal(mu, sigma, size=y_t.shape)
        x_t_noisy = x_t + noise_x
        y_t_noisy = y_t + noise_y
        noisy_trajectories.append((t, x_t_noisy, y_t_noisy))
    return noisy_trajectories


def generate_trajectories(number_of_trajectories, max_deg_x, max_deg_y):
    """Génère un polynôme pour x et un pour y et renvoie x(t) et y(t)"""
    trajectories = []
    for _ in range(number_of_trajectories):
        poly_x = generate_lagrange_polynomial(max_deg_x)
        poly_y = generate_lagrange_polynomial(max_deg_y)
        t = np.linspace(0, 500, SIM_DURATION)
        x_t = poly_x(t)
        y_t = poly_y(t)
        trajectories.append((t, x_t, y_t))
    return trajectories


def plot_trajectories(trajectories):
    """Affiche la trajectoire"""
    n = len(trajectories)
    rows = int(np.ceil(np.sqrt(n)))
    cols = int(np.ceil(n / rows))
    
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 5))
    axes = np.array(axes).flatten()
    
    for i, (t, x_t, y_t) in enumerate(trajectories):
        ax = axes[i]
        ax.plot(x_t, y_t)
        ax.set_xlabel("x(t)")
        ax.set_ylabel("y(t)")
        ax.set_title(f"Curve {i+1}")
        ax.grid()
    
    for j in range(n, len(axes)):
        fig.delaxes(axes[j])
    
    plt.tight_layout()
    plt.show()

C. Exécution

In [971]:
# trajectories = add_noise_to_trajectories(generate_trajectories(NUMBER_OF_TRAJ, MAX_DEG_X, MAX_DEG_Y), 0, 0.05)
# plot_trajectories(trajectories[:10])  # Plot only 10 trajectories


# 3. PAT

In [972]:
trajectory = add_noise_to_trajectories(generate_trajectories(1, MAX_DEG_X, MAX_DEG_Y), 0, 0.01)
# print(trajectory[0])

x0 = trajectory[0][1][0]
y0 = trajectory[0][2][0]
target_position = np.array([x0, y0])
# target_position = np.array([0, 0])


target_history = [target_position.copy()]
tracking_history = [tracking_position.copy()]

A. Outils

In [973]:
def calculate_radius(distance, angle):
    """Calcule le rayon d'un faisceau à une certaine distance de la source"""
    radius = distance * np.tan(angle)
    return radius


def calculate_distance(point1, point2):
    """Calcule la distance entre la cible et le centre du faisceau"""
    distance = np.linalg.norm(point1 - point2)
    return distance


def detect_target():
    """Teste si la cible est dans le rayon du faisceau"""
    distance = calculate_distance(tracking_position, target_position)
    return distance <= beam_radius

B. Calcul du parcours du faisceau

In [974]:
def initialise_spiral_parameters(ax, dir):
    """Initialise les paramètres d'un déplacement en spirale commençant sur l'axe ax dans la direction dir"""
    global initial_axis, axis, direction, side_length, current_length, initial_position, spiral_count
    initial_axis = ax
    axis = ax
    direction = dir
    side_length = 1
    current_length = 0
    spiral_count = -1


def move_sensor_spiral():
    """Déplace le capteur le long de la spirale"""
    global tracking_position, axis, direction, side_length, current_length, spiral_count
    point = tracking_position.copy()
    step = STEP_COEF * beam_radius * 0.5
    print("calcul step")
    print(beam_radius)
    print(step)
    
    if current_length == side_length - 1:
        spiral_count += 1

    if current_length < side_length: # si le segment de spirale n'est pas fini, on continue dessus
        current_length += 1
    else: # si l'on arrive sur un coin, on tourne
        if axis != initial_axis:
            direction *= -1
            side_length += 1
        axis = 1 - axis
        current_length = 1

    point[axis] += direction * step
    tracking_position = np.clip(point.copy(), 0, WINDOW_SIZE)


def check_neighbourhood():

    # print(beam_radius)
    center = tracking_position
    closest_point = center.copy()
    min_dist = calculate_distance(closest_point, target_position)

    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            center = tracking_position.copy()
            directional_vector = np.array([dx, dy])
            temp_position = center + beam_radius * directional_vector
            distance = calculate_distance(temp_position, target_position)
            if distance < min_dist:
                closest_point = temp_position.copy()
                min_dist = distance
                # print(min_dist)
    # print(closest_point)
    # print(tracking_position)
    # print(target_position)
    # print(beam_radius)
    return closest_point


C. Calcul des déplacements des éléments

In [975]:
def move_target():
    """Met à jour les position et vitesse de la cible"""
    global target_position
    global target_speed

    x, y = target_position
    vx, vy = target_speed

    new_x = x + vx*DT + random.gauss(0, NOISE*vx*DT)
    new_y = y + vy*DT + random.gauss(0, NOISE*vy*DT)
    new_vx = (new_x - x) / DT
    new_vy = (new_y - y) / DT
    
    target_position = np.array([new_x, new_y])
    target_speed = np.array([new_vx, new_vy])
    target_position = np.clip(target_position, 0, WINDOW_SIZE)
    target_speed = np.clip(target_speed, 0, WINDOW_SIZE)

    # Limiter la taille de l'historique pour éviter une surcharge mémoire
    if len(target_history) > 50:
        target_history.pop(0)
    target_history.append(target_position.copy())


def move_target_trajectory(frame, trajectory):

    global target_position
    new_x = trajectory[0][1][frame]
    new_y = trajectory[0][2][frame]
    target_position = np.array([new_x, new_y])
    target_position = np.clip(target_position, 0, WINDOW_SIZE)

    # Limiter la taille de l'historique pour éviter une surcharge mémoire
    if len(target_history) > 50:
        target_history.pop(0)
    target_history.append(target_position.copy())


def search_target(frame):
    """Met à jour la position d'un faisceau cherchant la cible"""
    global tracking_position, is_tracking, beam_radius, initial_position, spiral_count, current_length, side_length, axis
    # tracking_position = np.clip(search_path[frame], 0, WINDOW_SIZE)
    # move_sensor_spiral()
    # print("search")

    if len(tracking_history) > 50:
        tracking_history.pop(0)
    tracking_history.append(tracking_position.copy())

    
    if spiral_count == 5:
        current_length -= 1
        move_sensor_spiral()
        if len(tracking_history) > 50:
            tracking_history.pop(0)
        tracking_history.append(tracking_position.copy())
        side_length -= 2
        beam_radius *= RADIUS_INCREASE_COEF
        spiral_count = 0
        print("beam increase")
        print(beam_radius)
        # initialise_spiral_parameters(axis, direction)
    
    if detect_target():
        spiral_count = 0
        if beam_radius <= calculate_radius(DISTANCE, RESOLUTION):
            is_tracking = True
            print("found")
        else:
            beam_radius *= RADIUS_DECREASE_COEF
            print("decrease")
            initialise_spiral_parameters(axis, direction)
            initial_position = tracking_position.copy()
    else:
        move_sensor_spiral()
    


def track_target(frame):
    """Met à jour la position d'un faisceau traquant une cible qu'il a déjà trouvé"""
    global tracking_position, beam_radius
    
    if len(tracking_history) > 50:
        tracking_history.pop(0)
    tracking_history.append(tracking_position.copy())
    
    if not detect_target():
        # print("increase")
        beam_radius *= RADIUS_INCREASE_COEF
    else:
        if beam_radius >= calculate_radius(DISTANCE, RESOLUTION):
            beam_radius *= RADIUS_DECREASE_COEF
            # print("decrease2")
        tracking_position = check_neighbourhood()

    # if not detect_target():
    #     is_tracking = False
    #     # make_square_spiral_path(frame, beam_radius)
    #     # beam_radius *= RADIUS_INCREASE_COEF
    #     beam_radius = calculate_radius(DISTANCE, BEAM_ANGLE)
    #     initialise_spiral_parameters(axis, direction)


D. Mise à jour de l'animation

In [976]:
def update(frame):
    """Détermine l'état de la frame suivante"""
    global tracking_position, target_position, count_frame_found, count_frame_tracking
    # if frame >= 1000:
    #     return
    
    if frame % 2 == 0:
        move_target_trajectory(frame, trajectory)
        # move_target()
    if is_tracking:
        track_target(frame)
        count_frame_tracking += 1 
        if detect_target():
            count_frame_found += 1     
    else :
        search_target(frame)


    # Mise à jour des données des objets graphiques
    target_x, target_y = zip(*target_history)
    tracking_x, tracking_y = zip(*tracking_history)

    target_line.set_data(target_x, target_y)
    tracking_line.set_data(tracking_x, tracking_y)
    target_dot.set_data([target_position[0]], [target_position[1]])
    sensor_dot.set_data([sensor_position[0]], [sensor_position[1]])
    tracking_dot.set_data([tracking_position[0]], [tracking_position[1]])

    return target_line, target_dot, sensor_dot, tracking_line, tracking_dot,

E. Exécution

In [977]:

beam_radius = calculate_radius(DISTANCE, BEAM_ANGLE)
print(beam_radius)
initialise_spiral_parameters(axis, direction)
# make_square_spiral_path(0, beam_radius)

ani = animation.FuncAnimation(fig, update, frames=SIM_DURATION, interval=30, blit=True)
plt.show()

2.731512449218952
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
calcul step
2.731512449218952
1.7071952807618451
beam increase
5.463024898437904
decrease
decrease
calcul step
1.365756224609476
0.8535976403809226
calcul step
1.365756224609476
0.8535976403809226
c

KeyboardInterrupt: 

# 4. Prise en compte de la diffusion de l'eau


A. Génération de données, histogramme de valeurs et courbes COR

In [ ]:
from scipy.stats import gamma
from scipy.special import gammaln
from scipy.optimize import minimize

# Log-likelihood function to maximize
def log_likelihood(mean, alpha, yi):
    P = len(yi)
    return (P * alpha * np.log(alpha / mean)
        - P * gammaln(alpha)
        + (alpha - 1) * np.sum(np.log(yi))
        - (alpha / mean) * np.sum(yi))

# Maximum likelihood estimation of mean
def max_log_likelihood(alpha, yi):
    result = minimize(lambda m: -log_likelihood(m, alpha, yi), [1.0], method='BFGS')
    return result.x[0]

# Data generation function
def generate_data(alpha, P, n, m0, r):
    m1 = m0 * (1 + r)
    y0 = np.random.gamma(alpha, m0 / alpha, P * n)
    y1 = np.random.gamma(alpha, m1 / alpha, P * n)
    means0 = np.mean(y0.reshape(n, P), axis=1)
    means1 = np.mean(y1.reshape(n, P), axis=1)
    ml0 = np.array([max_log_likelihood(alpha, y0[i * P : (i + 1) * P]) for i in range(n)])
    ml1 = np.array([max_log_likelihood(alpha, y1[i * P : (i + 1) * P]) for i in range(n)])
    l0_0 = np.array([log_likelihood(m0, alpha, y0[i * P : (i + 1) * P]) for i in range(n)])
    l1_0 = np.array([log_likelihood(m1, alpha, y0[i * P : (i + 1) * P]) for i in range(n)])
    l0_1 = np.array([log_likelihood(m0, alpha, y1[i * P : (i + 1) * P]) for i in range(n)])
    l1_1 = np.array([log_likelihood(m1, alpha, y1[i * P : (i + 1) * P]) for i in range(n)])
    return l0_0, l1_0, l0_1, l1_1

# Histogram plotting function
def plot_histograms(l0, l1):
    diff_0 = l1[0] - l0[0]  # l1_0 - l0_0
    diff_1 = l1[1] - l0[1]  # l1_1 - l0_1
    plt.figure(figsize=(8, 6))
    plt.hist([diff_0, diff_1], bins=50, density=True, color=['blue', 'red'], alpha=0.7,
             label=["$l_{1,0} - l_{0,0}$", "$l_{1,1} - l_{0,1}$"])
    plt.axvline(0, color='black', linestyle='--', linewidth=1, label='Zero')
    plt.xlabel("Value")
    plt.ylabel("Frequency")
    plt.grid()
    plt.legend()
    plt.show()

# ROC curve plotting function
def plot_roc(diff_0, diff_1, num_thresholds=1000):
    thresholds = np.linspace(min(diff_0.min(), diff_1.min()), max(diff_0.max(), diff_1.max()), num_thresholds)
    p_fa = [(diff_0 >= t).mean() for t in thresholds]
    p_d = [(diff_1 >= t).mean() for t in thresholds]
    plt.figure(figsize=(8, 6))
    plt.plot(p_fa, p_d, color='blue', label='ROC Curve')
    plt.xscale('log')
    plt.xlabel(r'$ P_{fa} $')
    plt.ylim([0, 1.05])
    plt.xlim([1e-5, 1])
    plt.ylabel(r'$ P_{de} $')
    plt.grid()
    plt.legend()
    plt.show()

B. Test

In [ ]:
# Generate data and plot results
l0_0, l1_0, l0_1, l1_1 = generate_data(2, 10, 10000, 5, 0.5)
plot_histograms([l0_0, l0_1], [l1_0, l1_1])
plot_roc(l1_0 - l0_0, l1_1 - l0_1)

KeyboardInterrupt: 

# 5. Filtre de Kalman

A. Application du filtre

In [ ]:
def kalman_filter_trajectories(trajectories, meas_var_x, meas_var_y, process_var_x, process_var_y):
    """Filtre de Kalman pour les trajectoires"""
    filtered_trajectories = []
    dt = 0.4  # Intervalle de temps entre chaque point de mesure
    A = np.array([[1, dt], [0, 1]])  # Matrice d'état
    H = np.array([[1, 0]])  # Matrice d'observation

    for t, x_noisy, y_noisy in trajectories:
        x = np.array([[x_noisy[0]], [0]])
        y = np.array([[y_noisy[0]], [0]])
        P = np.eye(2)
        Q_x = np.array([[process_var_x, 0], [0, process_var_x]])
        Q_y = np.array([[process_var_y, 0], [0, process_var_y]])
        R_x = meas_var_x
        R_y = meas_var_y

        x_filtered, y_filtered = [], []

        for i in range(len(t)):
            x = A @ x
            P = A @ P @ A.T + Q_x

            y = A @ y
            P = A @ P @ A.T + Q_y

            K_x = P @ H.T / (H @ P @ H.T + R_x)
            x = x + K_x * (x_noisy[i] - H @ x)
            P = (np.eye(2) - K_x @ H) @ P
            x_filtered.append(x[0, 0])

            K_y = P @ H.T / (H @ P @ H.T + R_y)
            y = y + K_y * (y_noisy[i] - H @ y)
            P = (np.eye(2) - K_y @ H) @ P
            y_filtered.append(y[0, 0])

        filtered_trajectories.append((t, np.array(x_filtered), np.array(y_filtered)))

    return filtered_trajectories

B. Optimisation de paramètres

In [ ]:
def estimate_noise_variances(trajectories, smooth_sigma):
    """Estime les paramètres de bruit sur les trajectoires bruitées"""
    meas_vars_x, meas_vars_y = [], []
    process_vars_x, process_vars_y = [], []
    for t, x_t_noisy, y_t_noisy in trajectories:
        x_smooth = gaussian_filter1d(x_t_noisy, sigma=smooth_sigma)
        y_smooth = gaussian_filter1d(y_t_noisy, sigma=smooth_sigma)

        meas_vars_x.append(np.var(x_t_noisy - x_smooth))
        meas_vars_y.append(np.var(y_t_noisy - y_smooth))

        vx = np.gradient(x_smooth, t)
        vy = np.gradient(y_smooth, t)
        ax = np.gradient(vx, t)
        ay = np.gradient(vy, t)

        process_vars_x.append(np.var(ax))
        process_vars_y.append(np.var(ay))

    meas_var_x = np.mean(meas_vars_x)
    meas_var_y = np.mean(meas_vars_y)
    process_var_x = np.mean(process_vars_x)
    process_var_y = np.mean(process_vars_y)

    return meas_var_x, meas_var_y, process_var_x, process_var_y

def calculate_trajectory_error(trajectories, estimates):
    """Calcule l'erreur quadratique totale entre les trajectoires vraies et estimées"""    
    total_error = 0
    for (t_true, x_true, y_true), (t_est, x_est, y_est) in zip(trajectories, estimates):
        error = np.sum((x_true - x_est)**2 + (y_true - y_est)**2)
        total_error += error
    return total_error

def compute_error_for_sigma(trajectories_true, trajectories_noisy, sigma_smooth):
    """Calcule l'erreur pour un certain sigma_smooth de lissage"""
    meas_var_x, meas_var_y, process_var_x, process_var_y = estimate_noise_variances(trajectories_noisy, sigma_smooth)
    filtered_trajectories = kalman_filter_trajectories(trajectories_noisy, meas_var_x, meas_var_y, process_var_x, process_var_y)
    error = calculate_trajectory_error(trajectories_true, filtered_trajectories)
    return error

def grid_search(trajectories_true, trajectories_noisy, sigma_values):
    """Recherche du meilleur sigma_smooth pour minimiser l'erreur"""
    best_sigma = None
    best_error = float('inf')
    step = 0
    for sigma in sigma_values:
        step += 1
        print(step)
        error = compute_error_for_sigma(trajectories_true, trajectories_noisy, sigma)
        if error < best_error:
            best_error = error
            best_sigma = sigma
    return best_sigma, best_error

C. Application

In [ ]:
# Générer des trajectoires vraies
trajectories_true = generate_trajectories(n_trajectories, max_deg_x, max_deg_y)

# Ajouter du bruit aux trajectoires
mu = 0
sigma = 0.1
trajectories_noisy = add_noise_to_trajectories(trajectories_true, mu, sigma)

# Optimiser sigma avec grid search
sigma_values = np.linspace(8, 12, 100)
best_sigma, best_error = grid_search(trajectories_true, trajectories_noisy, sigma_values)

meas_var_x, meas_var_y, process_var_x, process_var_y = estimate_noise_variances(trajectories_noisy, best_sigma)

# Filtrer les trajectoires avec le filtre de Kalman et afficher
filtered_trajectories = kalman_filter_trajectories(trajectories_noisy, meas_var_x, meas_var_y, process_var_x, process_var_y)

# Afficher les trajectoires

# t_true, x_true, y_true = trajectories_true[0]
# t_noisy, x_noisy, y_noisy = trajectories_noisy[0]
# t_filtered, x_filtered, y_filtered = filtered_trajectories[0]
# plt.figure(figsize=(10, 6))
# plt.plot(x_true, y_true, label='Trajectoire vraie (y)', color='g')
# plt.scatter(x_noisy, y_noisy, label='Trajectoire bruitée (y)', color='r', s=5)
# plt.plot(x_filtered, y_filtered, label='Trajectoire filtrée (y)', color='b')
# plt.legend()
# plt.title(f"Trajectoire")
# plt.show()

NameError: name 'n_trajectories' is not defined